# Homework 1 - Part 3: Spectral Analysis of Fine-tuned Language Models

## Learning Objectives

In this assignment, you will learn how to:
1. Use **WeightWatcher** to analyze neural network weight matrices
2. Understand the **power law exponent (α)** as a measure of layer quality
3. Track how model quality evolves during fine-tuning
4. Interpret spectral analysis results

## Background: What is WeightWatcher?

**WeightWatcher** uses **Random Matrix Theory** to analyze the weight matrices in neural networks. It computes a metric called the **power law exponent (α)** for each layer.

### Understanding the Power Law Exponent (α)

Think of α as a "health score" for each layer:

- **α between 2 and 6**: ✅ **Healthy layer** - Well-trained, good generalization
- **α < 2**: ⚠️ **Over-trained** - Layer may be memorizing, not generalizing  
- **α > 6**: ⚠️ **Under-trained** - Layer needs more training

### Why is this useful?

Traditional metrics (like loss or accuracy) only tell you how well the model performs on data. The power law exponent tells you about the **internal quality** of the model:
- Are layers learning good representations or just memorizing?
- Which layers are well-trained vs problematic?
- Is fine-tuning improving or degrading the model?

## Assignment Tasks

1. **Coding Task 1**: Calculate perplexity from loss (1 line of code)
2. **Coding Task 2**: Create a simple visualization (2-3 lines of code)
3. **Interpretation Questions**: Answer guided questions about the results

Let's get started!


---
## Step 1: Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os
import warnings
import requests
import random
warnings.filterwarnings('ignore')

# HuggingFace Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from datasets import Dataset

# WeightWatcher
import weightwatcher as ww

print("✓ All libraries imported successfully")

---
## Step 2: Configuration

In [ ]:
# Visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Configuration (lighter for faster execution)
SEED = 42
MODEL_NAME = "distilgpt2"  # 82M parameter model
MAX_LENGTH = 256  # Shorter sequences for speed
NUM_EPOCHS = 1  # Just 1 epoch for speed
BATCH_SIZE = 8
MAX_SAMPLES = 500  # Limit dataset size for speed
OUTPUT_DIR = "./finetuned_model_light"

# Set seeds
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device: {device}")
print(f"Training {MODEL_NAME} for {NUM_EPOCHS} epoch with {MAX_SAMPLES} samples")

---
## Step 3: Load and Prepare Dataset

We'll use a subset of the **IR Triplets** dataset for faster training.

In [ ]:
# Download IR Triplets dataset
url = "https://raw.githubusercontent.com/omroot/InductiveSLM/master/cache/raw_data/ir_triplets/ir_triplets.json"
print(f"Downloading dataset...")
response = requests.get(url, timeout=20)
data = response.json()

# Take a subset for speed
random.seed(SEED)
data = random.sample(data, min(MAX_SAMPLES, len(data)))

# Split into train/test
test_size = int(0.2 * len(data))
test_data = data[:test_size]
train_data = data[test_size:]

print(f"✓ Loaded {len(train_data)} train + {len(test_data)} test examples")

# Format for causal LM (combine Context + Question + Answer)
def format_example(ex):
    text = f"Context: {ex['Training Observations']}\n\nQuestion: {ex['Question']}\n\nAnswer: {ex['Answer']}"
    return {"text": text}

train_formatted = [format_example(ex) for ex in train_data]
test_formatted = [format_example(ex) for ex in test_data]

print(f"✓ Formatted datasets")

---
## Step 4: Load Model and Tokenize

In [ ]:
# Load model and tokenizer
print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = model.to(device)
print(f"✓ Model loaded: {sum(p.numel() for p in model.parameters()):,} parameters")

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=MAX_LENGTH, padding="max_length")

train_dataset = Dataset.from_list(train_formatted).map(tokenize_function, batched=True, remove_columns=["text"])
test_dataset = Dataset.from_list(test_formatted).map(tokenize_function, batched=True, remove_columns=["text"])

print(f"✓ Datasets tokenized")

---
## Step 5: Analyze Pre-trained Model with WeightWatcher

In [ ]:
print("="*60)
print("ANALYZING PRE-TRAINED MODEL")
print("="*60)

# Run WeightWatcher
watcher = ww.WeightWatcher(model=model.cpu())
details_pre = watcher.analyze(plot=False)
model.to(device)  # Move back to device

# Extract metrics
alpha_values_pre = details_pre['alpha'].tolist()
alpha_mean_pre = np.mean(alpha_values_pre)
n_layers = len(alpha_values_pre)
n_good_layers_pre = sum(1 for a in alpha_values_pre if 2 <= a <= 6)

print(f"\n✓ Pre-trained model analysis:")
print(f"  Layers analyzed: {n_layers}")
print(f"  Mean α: {alpha_mean_pre:.3f}")
print(f"  Good layers (2 ≤ α ≤ 6): {n_good_layers_pre}/{n_layers}")

---
## Step 6: Fine-tune the Model

In [ ]:
print("="*60)
print("FINE-TUNING MODEL")
print("="*60)

os.makedirs(OUTPUT_DIR, exist_ok=True)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=20,
    weight_decay=0.01,
    logging_steps=20,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    seed=SEED,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

# Train
train_result = trainer.train()
print(f"\n✓ Training complete! Training loss: {train_result.training_loss:.4f}")

# Evaluate
test_results = trainer.evaluate(test_dataset)
test_loss = test_results['eval_loss']
print(f"  Test loss: {test_loss:.4f}")

---
## 🎯 CODING TASK 1: Calculate Perplexity

### Background

**Perplexity** measures how "surprised" the model is by the test data.
- **Lower perplexity** = Better model
- **Formula**: perplexity = exp(loss)

### Your Task

Complete the code below to calculate test perplexity.

**Hint**: Use `np.exp(test_loss)`

In [ ]:
# TODO: Calculate perplexity from test_loss
# Replace None with your code
test_perplexity = None  # YOUR CODE HERE

print(f"Test Perplexity: {test_perplexity:.2f}")
print(f"Interpretation: Lower is better. Perfect model = 1.0")

---
## Step 7: Analyze Fine-tuned Model

In [ ]:
print("="*60)
print("ANALYZING FINE-TUNED MODEL")
print("="*60)

# Run WeightWatcher on fine-tuned model
watcher_post = ww.WeightWatcher(model=model.cpu())
details_post = watcher_post.analyze(plot=False)
model.to(device)

# Extract metrics
alpha_values_post = details_post['alpha'].tolist()
alpha_mean_post = np.mean(alpha_values_post)
n_good_layers_post = sum(1 for a in alpha_values_post if 2 <= a <= 6)

print(f"\n✓ Fine-tuned model analysis:")
print(f"  Layers analyzed: {len(alpha_values_post)}")
print(f"  Mean α: {alpha_mean_post:.3f}")
print(f"  Good layers (2 ≤ α ≤ 6): {n_good_layers_post}/{len(alpha_values_post)}")

print(f"\nChange from pre-trained to fine-tuned:")
print(f"  Δα = {alpha_mean_post - alpha_mean_pre:+.3f}")

---
## Step 8: Create Summary

In [ ]:
# Create summary DataFrame
summary_data = [
    {
        'Stage': 'Pre-trained',
        'Mean α': f"{alpha_mean_pre:.3f}",
        'Good Layers': f"{n_good_layers_pre}/{n_layers}",
        'Test Loss': 'N/A',
        'Perplexity': 'N/A'
    },
    {
        'Stage': 'Fine-tuned',
        'Mean α': f"{alpha_mean_post:.3f}",
        'Good Layers': f"{n_good_layers_post}/{len(alpha_values_post)}",
        'Test Loss': f"{test_loss:.4f}",
        'Perplexity': f"{test_perplexity:.2f}" if test_perplexity is not None else 'N/A'
    }
]

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*60)
print("SUMMARY TABLE")
print("="*60)
print(summary_df.to_string(index=False))

---
## 🎯 CODING TASK 2: Create a Simple Bar Chart

### Background

Let's visualize how the mean α changed from pre-trained to fine-tuned.

### Your Task

Complete the code below to create a bar chart comparing the two α values.

**Hint**: Use `plt.bar(['Pre-trained', 'Fine-tuned'], [value1, value2])`

In [ ]:
# TODO: Create a bar chart comparing alpha_mean_pre and alpha_mean_post
# Replace None with your code
plt.figure(figsize=(8, 5))
plt.bar(None, None)  # YOUR CODE HERE: ['Pre-trained', 'Fine-tuned'], [alpha_mean_pre, alpha_mean_post]

plt.axhline(y=2, color='green', linestyle='--', alpha=0.5, label='Healthy range')
plt.axhline(y=6, color='green', linestyle='--', alpha=0.5)
plt.ylabel('Mean α', fontsize=12)
plt.title('Power Law Exponent Before and After Fine-tuning', fontsize=14)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('alpha_comparison.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Visualization saved as 'alpha_comparison.png'")

---
## 📝 INTERPRETATION QUESTIONS

Answer the following questions based on your results.

### Question 1: Understanding α

**Q1a**: What was the mean α for the pre-trained DistilGPT2 model?

**Your answer**: [Write your answer here]

**Q1b**: Is this in the "healthy" range (2-6)? What does this tell you?

**Your answer**: [Write your answer here]


### Question 2: Changes During Training

**Q2a**: Did the mean α increase, decrease, or stay approximately the same after fine-tuning?

**Your answer**: [Write your answer here]

**Q2b**: What do you think this means about the quality of fine-tuning?

**Your answer**: [Write your answer here]


### Question 3: Practical Application

**Q3**: Imagine you're training a model and you see α values dropping below 2 for many layers. Based on what you learned, what would this indicate and what might you do about it?

**Your answer**: [2-3 sentences]

*Hint: Remember what α < 2 means!*


---
## 🎉 Congratulations!

You've completed the spectral analysis assignment!

### What You Learned

1. ✅ How to use WeightWatcher to analyze models
2. ✅ What power law exponent (α) means and how to interpret it
3. ✅ How to calculate perplexity from loss
4. ✅ How to visualize spectral analysis results
5. ✅ Practical applications of this method

### How to Reuse This Method

You can use this same approach on ANY PyTorch model:

```python
import weightwatcher as ww

# Analyze any model!
watcher = ww.WeightWatcher(model=your_model)
details = watcher.analyze(plot=False)
alpha_values = details['alpha'].tolist()
mean_alpha = np.mean(alpha_values)
print(f"Mean α: {mean_alpha:.3f}")
```

### Submission

Submit this completed notebook with:
1. Both coding tasks completed
2. All interpretation questions answered
3. All cells executed (with outputs visible)

**Good luck!** 🚀